In [2]:
import pandas as pd
pd.set_option('display.max_columns', None)
import pyreadstat as st
import numpy as np
import matplotlib.pyplot as plt


path = r"C:\Users\HP\OneDrive\Escritorio\David Guzzi\DiTella\MEC\Materias\2025\2025 1T\[MT09] Econometría de Datos de Panel\Clases prácticas\PS 3-20250531\data\mod_abdata.dta"

df, meta = st.read_dta(path)
df.head(1)

,c1,ind,year,emp,wage,cap,indoutpt,n,w,k,ys,rec,yearm1,id,nL1,nL2,wL1,kL1,kL2,ysL1,ysL2,yr1977,yr1978,yr1979,yr1980,yr1981,yr1982
0,1-1,7.0,1977.0,5.041,13.1516,0.5894,95.707199,1.617604,2.576543,-0.52865,4.561294,1.0,1977.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0,0,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 828 entries, 0 to 827
Data columns (total 27 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   c1        828 non-null    object 
 1   ind       828 non-null    float64
 2   year      828 non-null    float64
 3   emp       828 non-null    float64
 4   wage      828 non-null    float64
 5   cap       828 non-null    float64
 6   indoutpt  828 non-null    float64
 7   n         828 non-null    float64
 8   w         828 non-null    float64
 9   k         828 non-null    float64
 10  ys        828 non-null    float64
 11  rec       828 non-null    float64
 12  yearm1    828 non-null    float64
 13  id        828 non-null    float64
 14  nL1       770 non-null    float64
 15  nL2       632 non-null    float64
 16  wL1       770 non-null    float64
 17  kL1       770 non-null    float64
 18  kL2       632 non-null    float64
 19  ysL1      770 non-null    float64
 20  ysL2      632 non-null    float6

In [5]:
df_reg = df.sort_values(['id', 'year'])
df_reg.dropna(subset=['n', 'nL1'], inplace=True)

import statsmodels.api as sm

# Variables
X = sm.add_constant(df_reg['nL1'])
y = df_reg['n']

# Estimar el modelo
model = sm.OLS(y, X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                      n   R-squared:                       0.990
Model:                            OLS   Adj. R-squared:                  0.990
Method:                 Least Squares   F-statistic:                 7.654e+04
Date:                Sat, 31 May 2025   Prob (F-statistic):               0.00
Time:                        21:05:54   Log-Likelihood:                 450.02
No. Observations:                 770   AIC:                            -896.0
Df Residuals:                     768   BIC:                            -886.8
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0379      0.006     -5.948      0.0

In [9]:
from statsmodels.iolib.summary2 import summary_col

# Imprimir estilo compacto
print(summary_col([model], stars=True, model_names=['t1']))


                   t1    
-------------------------
const          -0.0379***
               (0.0064)  
nL1            0.9967*** 
               (0.0036)  
R-squared      0.9901    
R-squared Adj. 0.9901    
Standard errors in
parentheses.
* p<.1, ** p<.05,
***p<.01


In [10]:
#FE

#1. LSDV
import statsmodels.formula.api as smf

# Estimación con fórmula estilo patsy: C(id) genera dummies para 'id'
model = smf.ols('n ~ nL1 + C(id)', data=df_reg).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                      n   R-squared:                       0.992
Model:                            OLS   Adj. R-squared:                  0.991
Method:                 Least Squares   F-statistic:                     585.9
Date:                Sat, 31 May 2025   Prob (F-statistic):               0.00
Time:                        21:10:42   Log-Likelihood:                 545.92
No. Observations:                 770   AIC:                            -813.8
Df Residuals:                     631   BIC:                            -168.0
Df Model:                         138                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          0.1133      0.081      1.

In [ ]:
#2. FE
from linearmodels.panel import PanelOLS

df_fe = df.sort_values(['id', 'year'])
df_fe= df_fe.set_index(['id', 'year'])

# Variable dependiente y explicativa
y = df_fe['n']
X = df_fe[['nL1']]

# Modelo con efectos fijos por entidad (id)
model_fe = PanelOLS(y, X, entity_effects=True).fit()

print(model_fe.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:                      n   R-squared:                        0.4926
Estimator:                   PanelOLS   R-squared (Between):              0.9850
No. Observations:                 770   R-squared (Within):               0.4926
Date:                Sat, May 31 2025   R-squared (Overall):              0.9806
Time:                        21:13:27   Log-likelihood                    545.92
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      612.49
Entities:                         138   P-value                           0.0000
Avg Obs:                       5.5797   Distribution:                   F(1,631)
Min Obs:                       5.0000                                           
Max Obs:                       6.0000   F-statistic (robust):             612.49
                            

c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\linearmodels\panel\model.py:1260: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)


In [12]:
# IV2SLS

df_iv2sls = df.sort_values(['id', 'year'])

# Crear diferencias
df_iv2sls['Dn'] = df_iv2sls.groupby('id')['n'].diff()
df_iv2sls['DnL1'] = df_iv2sls.groupby('id')['nL1'].diff()

# Eliminar filas con NaN
df_iv2sls = df_iv2sls.dropna(subset=['Dn', 'DnL1'])


from linearmodels.iv import IV2SLS

# Definir variables
y = df_iv2sls['Dn']
X = None                     # No exógenas distintas del intercepto
endog = df_iv2sls['DnL1']        # Variable endógena
instr = df_iv2sls['nL2']         # Instrumento

# Modelo sin constante (ya que estamos en diferencias)
model = IV2SLS(dependent=y, exog=X, endog=endog, instruments=instr).fit()
print(model.summary)

                          IV-2SLS Estimation Summary                          
Dep. Variable:                     Dn   R-squared:                     -2.0842
Estimator:                    IV-2SLS   Adj. R-squared:                -2.0890
No. Observations:                 632   F-statistic:                    18.200
Date:                Sat, May 31 2025   P-value (F-stat)                0.0000
Time:                        21:20:39   Distribution:                  chi2(1)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
DnL1           2.0156     0.4725     4.2662     0.0000      1.0896      2.9416

Endogenous: DnL1
Instruments: nL2
Robust Covariance